# Classification of trajectories 
Here we classify trajectory data based on the training datasets of labelled trajectories.  

In [1]:
import numpy as np
import torch
import os
import sys
sys.path.append('../')

from src.features import Q_measure, dist_distribution, convex_hull
from src.andi_tools import Andi_to_xy
from src.training import FeedForward

%load_ext autoreload
%autoreload 2

In [5]:
# load trajectories from training dataset with labels  
trajectories, labels = Andi_to_xy("data/test_2d.csv","data/label_2d.csv")

# make labels in format we need 
labels = torch.FloatTensor(labels)
labels = torch.max(labels, 1)[1]

## Features of trajectories calculation 
List of all features is available in source code. We estimate Q measure of trajectories for different window size,
distanceof jumps distributions. 

Comment: Q measure with different window size parameters gives redundant information about trajectory.



In [6]:
# estimate features of data-trajectory: e.g. Q measure with parameter of window size w

w1 = 2 #parameter of window size  
w2 = 4 #parameter of window size  
w3 = 6 #parameter of window size  

# run features function which calculates different features 
features =[Q_measure(trajectories, w1),
            Q_measure(trajectories, w2),
            Q_measure(trajectories, w3),
          dist_distribution(trajectories)]

# make features to be of the right size 
feature_sizes = [f.shape[1] for f in features]
features = [torch.FloatTensor(f) for f in features]


# Training of neural network
Model of training of neural network used: feed forward. 

We use cross entropy. 
We use Adam optimizer function for its robustness. The learning rate parameter lr is chosen by default to be lr=0.001. However performance of NN depends on it. 

In [7]:
# function for training network with certain parameters 

def train_net(lr_i):
    model = FeedForward(feature_sizes)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = lr_i) # learning rate default is 0.01 

    model.train()
    epoch = 1000
    for epoch in range(epoch):    
        optimizer.zero_grad()    # Forward pass
        y_pred = model(features)    # Compute Loss
        loss = criterion(y_pred.squeeze(), labels)
   
        #print('Epoch {}: train loss: {}'.format(epoch, loss.item()))    # Backward pass
        loss.backward()
        optimizer.step()


We want to see how learning rate influences neural network performance. 
In general, the learning rate is a hyperparameter that controls how much to change the model in response to the estimated error each time the model weights are updated. Choosing the learning rate is challenging as a value too small may result in a long training process that could get stuck.

In [14]:
lr_i = 0.001
model = FeedForward(feature_sizes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) # learning rate default is 0.01 

train_net(lr_i)

for ind in range(1,5):
    lr_i = 1./ind
    train_net(lr_i)
    print('learning rate', lr_i)
    
    
    model.train()
    epoch = 1000
    for epoch in range(epoch):    
        optimizer.zero_grad()    # Forward pass
        y_pred = model(features)    # Compute Loss
        loss = criterion(y_pred.squeeze(), labels)
   
    #print('Epoch {}: train loss: {}'.format(epoch, loss.item()))    # Backward pass
    loss.backward()
    optimizer.step()
    
    
    
    after_train = criterion(y_pred,labels)
    prediction = torch.max(y_pred, 1)[1]

    
    nonzero = np.count_nonzero(np.argmax(y_pred.detach().numpy(),axis=1)-labels.detach().numpy())
    tot = len(np.argmax(y_pred.detach().numpy(),axis=1)-labels.detach().numpy())

    print((tot-nonzero)/tot)
    (tot-nonzero)/tot

learning rate 1.0
0.256
learning rate 0.5
0.256
learning rate 0.3333333333333333
0.256
learning rate 0.25
0.256


# Model training with one parameter setting
Previous simulations and model training 

In [10]:
model = FeedForward(feature_sizes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) # learning rate default is 0.01 




In [11]:
model.train()
epoch = 1000
for epoch in range(epoch):    
    optimizer.zero_grad()    # Forward pass
    y_pred = model(features)    # Compute Loss
    loss = criterion(y_pred.squeeze(), labels)
   
    #print('Epoch {}: train loss: {}'.format(epoch, loss.item()))    # Backward pass
    loss.backward()
    optimizer.step()

In [12]:
after_train = criterion(y_pred,labels)

In [15]:
prediction = torch.max(y_pred, 1)[1]

In [16]:
nonzero = np.count_nonzero(np.argmax(y_pred.detach().numpy(),axis=1)-labels.detach().numpy())
tot = len(np.argmax(y_pred.detach().numpy(),axis=1)-labels.detach().numpy())

In [17]:
(tot-nonzero)/tot

# change the optimiser of the model and its parameters above 
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
#0.456 for lr=0.01
#0.33 for lr=0.001

0.33